In [1]:
!pip install mediapipe
!pip install opencv-python
!pip install PyAutoGUI

You should consider upgrading via the 'C:\Users\Denis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Denis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Denis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import mediapipe as mp
import pyautogui

In [5]:
def show_webcam():
    scale = 10

    camera = cv2.VideoCapture(0)

    face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
    screen_w, screen_h = pyautogui.size()

    while True:
        _, frame = camera.read()
        frame = cv2.flip(frame, 1)

        #get the webcam size
        height, width, channels = frame.shape
        #prepare the crop 
        centerX,centerY=int(height/2),int(width/2)
        radiusX,radiusY= int(scale*height/50),int(scale*width/50)
        minX,maxX=centerX-radiusX,centerX+radiusX
        minY,maxY=centerY-radiusY,centerY+radiusY
        cropped = frame[minX:maxX, minY:maxY]
        resized_cropped = cv2.resize(cropped, (width, height))

        rgb_frame = cv2.cvtColor(resized_cropped, cv2.COLOR_BGR2RGB)
        output = face_mesh.process(rgb_frame)
        landmark_points = output.multi_face_landmarks
        # print(landmark_points)
        frame_h, frame_w, _ = resized_cropped.shape
        if landmark_points:
            landmarks = landmark_points[0].landmark
            for id, landmark in enumerate(landmarks[473:478]):
                x = int(landmark.x * frame_w)
                y = int(landmark.y * frame_h)
                cv2.circle(resized_cropped, (x, y), 3, (0, 255, 0))
                if id == 1:
                    screen_x = screen_w / frame_w * x
                    screen_y = screen_h / frame_h * y
                    try:
                        pyautogui.moveTo(screen_x, screen_y) # Any PyAutoGUI (without side effects) call will do here.
                    except pyautogui.FailSafeException:
                        print('Running code before exiting.') # Your code here.
                        break
                    pyautogui.moveTo(screen_x, screen_y)
            left = [landmarks[145], landmarks[159]]
            for landmark in left:
                x = int(landmark.x * frame_w)
                y = int(landmark.y * frame_h)
                cv2.circle(resized_cropped, (x, y), 3, (0, 255, 255))
                print(left[0].y - left[1].y)
            if (left[0].y - left[1].y) < 0.04:
                # print('click')
                pyautogui.click()
                pyautogui.sleep(1)

        cv2.imshow('Eyes Controlled Mouse', resized_cropped)
        key = cv2.waitKey(10)
        # if Esc key is press then break out of the loop 
        if key == 27: 
            break
    
    camera.release()
    cv2.destroyAllWindows()

def main():
    show_webcam()

if __name__ == '__main__':
    main()
